In [2]:
import networkx as nx
import json
import numpy as np
import pandas as pd

from scripts.search_utils import build_graph_from_async
from scripts.process_utils import (compare_results_with_askcos,
                                   print_rxns,print_bypass_rxns,
                                   print_rxns_list)
from scripts.utils import *

from rdkit.Chem import AllChem as Chem
from rdkit import DataStructs

import urllib.parse
from IPython.display import Image,display

project_directory:  /home/xuan/GitLab/synthetic-field-score


In [2]:
def enzy_template_id_search(template_id,target_mol):
    """
    Searches for templates matching a given template ID and finds the most similar molecule to a target molecule.

    Args:
    template_id (str/int): The template ID to search for in the template database.
    target_mol (str): The SMILES representation of the target molecule.

    Returns:
    DataFrame Row: The row from the template DataFrame corresponding to the most similar molecule.
    """
    
    templates = pd.read_json('/home/xuan/GitLab/hybmind/data/bkms-retro.templates.bkms.json.gz')
    template_df = pd.read_json('/home/xuan/GitLab/hybmind/data/bkms-templates.df.json.gz')
    index = int(templates[templates['_id']==template_id]['index'])
    
    ref_mol = Chem.MolFromSmiles(target_mol)
    ref_fp = Chem.GetMorganFingerprintAsBitVect(ref_mol, 2, nBits=1024)

    new_template_df = template_df.loc[template_df['index']==index,['products']]
    mol_smarts_list = [new_template_df.loc[idx,'products'] for idx in new_template_df.index]
    mol_list = [Chem.MolFromSmiles(mol_smarts) for mol_smarts in mol_smarts_list]

    fp_list = [Chem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024) for mol in mol_list]
    sim_list = [DataStructs.TanimotoSimilarity(ref_fp,fp) for fp in fp_list]
    largest_smi_index = np.argmax(sim_list)
    return template_df.loc[new_template_df.index[largest_smi_index]]

# Process search results of 1001 molecules from ZINC "boutique" subset

In [2]:
shortest_path_dict = {}
spg_dict = {}

prioritizer_list = ['async','0.15sync','fh']
for i in range(1001):
    for prioritizer in prioritizer_list:
        path = f'./results/zinc_result/boutique_{prioritizer}_{i}.json'
        with open(path, "r") as outfile:
            result_dict = json.load(outfile)
        smiles = result_dict['start_node']['id']
        if prioritizer == 'async':
            shortest_path_dict[smiles] = {}
            spg_dict[smiles] = {}

        graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
        try:
            g = nx.node_link_graph(graph_json)
            if g and len(g.nodes)>1:
                std_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
                chem_nodes = [n for n in g.nodes if '>>' not in n]
                starting_nodes = [n for n in chem_nodes if g.nodes[n]['terminal']]
                s_p_g = metabolic_dijkstra(g, starting_nodes)
                spg_dict[smiles][prioritizer] = s_p_g
                try:
                    shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[std_smiles]['path_length']
                except:
                    shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[smiles]['path_length']
            else:
                print ('No Graph for', smiles, prioritizer)
        except:
            print ('Graph retrieval failed for', smiles, f'boutique_{i}')

No Graph for COCOC(=O)[C@@H]1N2C(=O)[C@@H](N3C(=O)[C@H](c4ccccc4)NC3(C)C)[C@H]2SC1(C)C 0.15sync
No Graph for O=C1NCCc2cc(Br)c(O)c(c2)Oc2ccc(cc2Br)[C@H](O)CNC(=O)/C(=N/O)Cc2cc(Br)c(O)c(c2)Oc2ccc(cc2Br)C/C1=N\O 0.15sync
No Graph for CC(=O)OC[C@]12[C@H](OC(C)=O)[C@H](OC(C)=O)[C@@H]3[C@@H](OC(C)=O)[C@@]14O[C@@]3(C)COC(=O)c1cccnc1CC[C@@](C)(OC(=O)c1ccccc1)C(=O)O[C@@H]([C@H](OC(C)=O)[C@@H]2OC(C)=O)[C@]4(C)O 0.15sync
No Graph for CN1[C@H]2C[C@H](OC(=O)[C@H](CO)c3ccccc3)C[C@@H]1[C@@H]1O[C@H]12 0.15sync
No Graph for COc1c(OC)c(OC)c2c3c(cc4ccccc42)NC(=O)c13 0.15sync
No Graph for COCCO[C@H](C)OCc1ccccc1 0.15sync
No Graph for O[C@]12C=C(c3nccc4c3[nH]c3ccccc34)[C@@H]3CCN(CCCC/C=C/CC1)C[C@@]31C[C@@H]3C=CCCCCN3[C@@H]21 0.15sync
No Graph for C=CCNS(=O)(=O)c1cc(S(N)(=O)=O)c(Cl)cc1/N=C/C=C(/C)O 0.15sync
No Graph for CC(=O)OC[C@]12[C@H](OC(C)=O)[C@H](OC(C)=O)[C@@H]3[C@@H](OC(C)=O)[C@@]14O[C@@]3(C)COC(=O)c1cccnc1[C@H](C)[C@H](C)C(=O)O[C@@H]([C@H](OC(C)=O)[C@@H]2OC(C)=O)[C@]4(C)O 0.15sync
No Graph for CCCC

In [3]:
shortest_paths = pd.DataFrame(shortest_path_dict).T    
any_path_found = shortest_paths.copy()
for idx in shortest_paths.index:
    for col in shortest_paths.columns:
        addl_data = shortest_paths.loc[idx, col]
        if addl_data < np.inf:
            any_path_found.loc[idx, col] = 1
        elif addl_data > 12:
            any_path_found.loc[idx, col] = 0
any_path_found = any_path_found.dropna(axis=0)
any_path_found = any_path_found>0
for prioritizer in prioritizer_list:
    shortest_paths[prioritizer] = shortest_paths[prioritizer].fillna(np.inf)
shortest_paths

,async,0.15sync,fh
CC(=O)[C@@](C)(O)C(=O)O,1.0,1.0,1.0
CC(=O)NCCC=O,1.0,1.0,1.0
CCCOC(=O)[C@@H](C)S,2.0,2.0,2.0
C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@]2(C)[C@@]1(O)C(=O)COCc1ccccc1,1.0,1.0,1.0
Nc1nc2c(ncn2CC[C@@H](O)CO)c(=O)[nH]1,3.0,2.0,2.0
...,...,...,...
CCC(=O)O[C@@]1(c2ccccc2)C[C@H](C)N(C)C[C@@H]1C,5.0,5.0,inf
C/C=C/C(=O)c1c(OC)cc2cc(OC)cc(OC)c2c1OC,inf,inf,inf
Cc1ccc([C@H](O)[C@@H](C)NCCCc2ccccc2)cc1O,3.0,3.0,inf
N[C@H](CO)C(=O)N[C@H](CC(=O)O)C(=O)O,1.0,1.0,1.0


In [4]:
askcos_shortest_paths = pd.read_csv('./data/askcos_shortest_paths.csv',index_col=0)
merged_pd = shortest_paths.join(askcos_shortest_paths)
merged_pd

,async,0.15sync,fh,"bkms,reaxys",bkms,reaxys
CC(=O)[C@@](C)(O)C(=O)O,1.0,1.0,1.0,1.0,1.0,5.0
CC(=O)NCCC=O,1.0,1.0,1.0,1.0,1.0,1.0
CCCOC(=O)[C@@H](C)S,2.0,2.0,2.0,3.0,2.0,3.0
C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@]2(C)[C@@]1(O)C(=O)COCc1ccccc1,1.0,1.0,1.0,inf,inf,inf
Nc1nc2c(ncn2CC[C@@H](O)CO)c(=O)[nH]1,3.0,2.0,2.0,3.0,inf,3.0
...,...,...,...,...,...,...
CCC(=O)O[C@@]1(c2ccccc2)C[C@H](C)N(C)C[C@@H]1C,5.0,5.0,inf,inf,inf,7.0
C/C=C/C(=O)c1c(OC)cc2cc(OC)cc(OC)c2c1OC,inf,inf,inf,9.0,inf,inf
Cc1ccc([C@H](O)[C@@H](C)NCCCc2ccccc2)cc1O,3.0,3.0,inf,inf,inf,4.0
N[C@H](CO)C(=O)N[C@H](CC(=O)O)C(=O)O,1.0,1.0,1.0,1.0,1.0,2.0


In [5]:
for prioritizer in prioritizer_list+['bkms,reaxys']:
    found = 0
    for index in merged_pd.index:
        tbtf_path_lenth = merged_pd.loc[index, prioritizer]
        if tbtf_path_lenth < np.inf:
            found += 1
    print(f'{prioritizer} molecule found: {found}, rate: {found/len(merged_pd):.2%}')

async molecule found: 720, rate: 71.93%
0.15sync molecule found: 683, rate: 68.23%
fh molecule found: 597, rate: 59.64%
bkms,reaxys molecule found: 493, rate: 49.25%


In [6]:
for prioritizer in prioritizer_list:
    compare_results_with_askcos(merged_pd,prioritizer)
    print('\n')

Totol number of molecules: 1001 
async found: 720 
ASKCOS found: 493 
async found but ASKCOS not found: 254 
async not found but ASKCOS found: 27 
Both async and ASKCOS found: 466 
async found shorter: 167 
async found equiv: 260 
async found longer: 39

1 step shorter: 100 
2 steps shorter: 48 
3 steps shorter: 13 
>=4 steps shorter: 6


Totol number of molecules: 1001 
0.15sync found: 683 
ASKCOS found: 493 
0.15sync found but ASKCOS not found: 226 
0.15sync not found but ASKCOS found: 36 
Both 0.15sync and ASKCOS found: 457 
0.15sync found shorter: 170 
0.15sync found equiv: 249 
0.15sync found longer: 38

1 step shorter: 106 
2 steps shorter: 46 
3 steps shorter: 13 
>=4 steps shorter: 5


Totol number of molecules: 1001 
fh found: 597 
ASKCOS found: 493 
fh found but ASKCOS not found: 162 
fh not found but ASKCOS found: 58 
Both fh and ASKCOS found: 435 
fh found shorter: 164 
fh found equiv: 248 
fh found longer: 23

1 step shorter: 97 
2 steps shorter: 49 
3 steps shorter: 13 
>

# Count how many chemical/reaction nodes in returned graphs

In [5]:
prioritizer_list = ['async','0.15sync','fh']
for prioritizer in prioritizer_list:
    num_chemical_list = []
    num_rxn_list = []
    num_enzy_rxn_list = []
    num_chem_rxn_list = []
    for i in range(1001):
    
        path = f'./results/zinc_result/boutique_{prioritizer}_{i}.json'
        with open(path, "r") as outfile:
            result_dict = json.load(outfile)
            target = list(result_dict['explored_rxns'].keys())[0]
        smiles = target
        graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
        g = nx.node_link_graph(graph_json)
        num_rxn = 0
        num_chem_rxn = 0
        num_enzy_rxn = 0
        num_chemical = 0
        for node in list(g.nodes):
            if '>>' in node:
                num_rxn += 1
                if 'bkms' in g.nodes[node]['model']:
                    num_enzy_rxn += 1
                elif 'RXN4Chem' in g.nodes[node]['model']:
                    num_chem_rxn += 1
            else:
                num_chemical += 1
        num_chemical_list.append(num_chemical)
        num_rxn_list.append(num_rxn)
        num_enzy_rxn_list.append(num_enzy_rxn)
        num_chem_rxn_list.append(num_chem_rxn)
    count_data = {'Chemical nodes':num_chemical_list,
                    'Reaction nodes':num_rxn_list,
                    'Enzy reaction nodes': num_enzy_rxn_list,
                    'Chem reaction nodes': num_chem_rxn_list}
    count_data_df = pd.DataFrame.from_dict(count_data)
    count_data_df.to_csv(f'./results/count_data/{prioritizer}_search_1000_nodes_count.csv',index=False)
    print('Saved csv file')

Saved csv file
Saved csv file
Saved csv file


# Show shorter pathways compared with "bkms,reaxys" method

In [22]:
def get_model_used(spg_dict, smiles, prioritizer = 'async'):
    """
    Retrieves and prints the model used for each path in the shortest pathway for a given SMILES.

    Args:
    spg_dict (dict): A dictionary containing pathway graph information.
    smiles (str): The molecule SMILES.
    prioritizer (str, optional): The method used to select the pathway. Defaults to 'async'.

    The function iterates through each path in the shortest pathway of the given SMIles, as determined by the prioritizer,
    and prints the model associated with each path.
    """
    for path in spg_dict[smiles][prioritizer].nodes[smiles]['shortest_pathway']:
        model = spg_dict[smiles][prioritizer].nodes[path]['model']
        print(model)

def get_shorter_list(shorter_num):
    """
    Generates a list of indices where the path length difference between two methods meets a specific criterion.

    Args:
    shorter_num (int): The specific number representing the difference in path length to filter the indices.

    Returns:
    list: A list of indices where the difference in path lengths between 'async' and 'bkms,reaxys' methods is equal to shorter_num.

    The function iterates through the indices of a merged Pandas DataFrame 'merged_pd'. It checks if the path lengths
    for 'async' and 'bkms,reaxys' methods are finite and if their difference equals shorter_num. If so, the index is appended to the list.
    """
    shorter_list = []
    for index in merged_pd.index:
        async_path_length = merged_pd.loc[index, 'async']
        askcos_path_length = merged_pd.loc[index, 'bkms,reaxys']
        if async_path_length < np.inf and askcos_path_length < np.inf:
            if askcos_path_length - async_path_length == shorter_num:
                shorter_list.append(index)
    return shorter_list

async_shorter_list= get_shorter_list(2)
print(len(async_shorter_list))
get_model_used(spg_dict, 'CC[C@H](C)Cn1c(=O)n(C)c(=O)c2[nH]c(C)nc21')

48

## (S)-verofylline

In [73]:
# 1 step shorter example
smi = 'CC[C@H](C)Cn1c(=O)n(C)c(=O)c2[nH]c(C)nc21'
print_rxns(spg_dict,'async',smi)

Product SFSCore: [0.566 0.435] ['bkms'] :  Cn1c(N)nc(N)c(N)c1=O>>Cn1c(=O)[nH]c(N)c(N)c1=O


Product SFSCore: [0.581 0.419] ['RXN4Chem'] :  CC[C@H](C)CBr.CN(C)C=O.Cn1c(=O)[nH]c(N)c(N)c1=O.O=C([O-])O.[K+].[K+]>>CC[C@H](C)Cn1c(N)c(N)c(=O)n(C)c1=O


Product SFSCore: [0.587 0.412] ['RXN4Chem'] :  CC(=O)O.CC[C@H](C)Cn1c(N)c(N)c(=O)n(C)c1=O>>CC[C@H](C)Cn1c(=O)n(C)c(=O)c2[nH]c(C)nc21


In [79]:
rxn = 'Cn1c(N)nc(N)c(N)c1=O>>Cn1c(=O)[nH]c(N)c(N)c1=O'
tform = spg_dict[smi]['async'].nodes[rxn]['tforms'][0]
enzy_template_id_search(tform,'Cn1c(=O)[nH]c(N)c(N)c1=O')


ID                                                                  15512
EC_Number                                                            None
Recommended_Name                                                     None
Reaction                2,5-Diamino-6-(5-phospho-D-ribitylamino)pyrimi...
Reaction_ID_BRENDA                                                   None
Reaction_ID_KEGG                                                   R09377
Reaction_ID_MetaCyc                                             RXN-10058
Reaction_ID_SABIO_RK                                                 None
BRENDA_Pathway_Name                                                  None
KEGG_Pathway_ID                                                   rn00740
KEGG_Pathway_Name                                   Riboflavin metabolism
MetaCyc_Pathway_ID                                     PWY-6168; PWY-6167
MetaCyc_Pathway_Name    flavin biosynthesis III (fungi); flavin biosyn...
Stoichiometry_Check                   

## (3S)-3-Hydroxy-β-ionone

In [80]:
# 2 steps shorter example
smi = 'CC(=O)/C=C/C1=C(C)C[C@H](O)CC1(C)C'
print_rxns(spg_dict,'async',smi)

Product SFSCore: [0.479 0.549] ['bkms'] :  CC(=O)CCC1=C(C)CCCC1(C)C>>CC(=O)/C=C/C1=C(C)CCCC1(C)C


Product SFSCore: [0.453 0.582] ['bkms'] :  CC(=O)/C=C/C1=C(C)CCCC1(C)C>>CC(=O)/C=C/C1=C(C)C[C@H](O)CC1(C)C


In [82]:
rxn = 'CC(=O)CCC1=C(C)CCCC1(C)C>>CC(=O)/C=C/C1=C(C)CCCC1(C)C'
tform = spg_dict[smi]['async'].nodes[rxn]['tforms'][0]
enzy_template_id_search(tform,'CC(=O)/C=C/C1=C(C)CCCC1(C)C')

ID                                                                  62312
EC_Number                                                        1.3.1.48
Recommended_Name           13,14-dehydro-15-oxoprostaglandin 13-reductase
Reaction                11alpha-hydroxy-9,15-dioxoprostanoate + NADP+ ...
Reaction_ID_BRENDA                                                BR36830
Reaction_ID_KEGG                                                     None
Reaction_ID_MetaCyc                                                  None
Reaction_ID_SABIO_RK                                                 None
BRENDA_Pathway_Name                                                  None
KEGG_Pathway_ID                                                      None
KEGG_Pathway_Name                                                    None
MetaCyc_Pathway_ID                                                   None
MetaCyc_Pathway_Name                                                 None
Stoichiometry_Check                   

In [83]:
rxn = 'CC(=O)/C=C/C1=C(C)CCCC1(C)C>>CC(=O)/C=C/C1=C(C)C[C@H](O)CC1(C)C'
tform = spg_dict[smi]['async'].nodes[rxn]['tforms'][0]
enzy_template_id_search(tform,'CC(=O)/C=C/C1=C(C)C[C@H](O)CC1(C)C')

ID                                                                  13906
EC_Number                                                      1.14.14.76
Recommended_Name                         ent-isokaurene C2/C3-hydroxylase
Reaction                ent-isokaurene + O2 + [reduced NADPH-hemoprote...
Reaction_ID_BRENDA                                                BR52394
Reaction_ID_KEGG                                                     None
Reaction_ID_MetaCyc                                                  None
Reaction_ID_SABIO_RK                                                 None
BRENDA_Pathway_Name                                                  None
KEGG_Pathway_ID                                                      None
KEGG_Pathway_Name                                                    None
MetaCyc_Pathway_ID                                                   None
MetaCyc_Pathway_Name                                                 None
Stoichiometry_Check                   

## dimenoxadol

In [49]:
# 2 steps shorter example
smi = 'CCOC(C(=O)OCCN(C)C)(c1ccccc1)c1ccccc1'
print_rxns(spg_dict,'async',smi)

Product SFSCore: [0.583 0.417] ['RXN4Chem'] :  CCO.O=C(O)C(O)(c1ccccc1)c1ccccc1.O=S(=O)(O)O>>CCOC(C(=O)O)(c1ccccc1)c1ccccc1


Product SFSCore: [0.606 0.391] ['RXN4Chem'] :  CCOC(C(=O)O)(c1ccccc1)c1ccccc1.CN(C)CCO>>CCOC(C(=O)OCCN(C)C)(c1ccccc1)c1ccccc1


# Case study: Ethambutol

In [114]:
scs_shortest_path_dict = {}
scs_spg_dict = {}
smi = 'CC[C@@H](CO)NCCN[C@@H](CC)CO'
name = 'Ethambutol'

path = f'./results/case_study/async_{name}.json'
with open(path, "r") as outfile:
    result_dict = json.load(outfile)


smiles = result_dict['start_node']['id']
scs_shortest_path_dict[smiles] = {}
scs_spg_dict[smiles] = {}
prioritizer = 'async'

graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
try:
    g = nx.node_link_graph(graph_json)
    if g and len(g.nodes)>1:
        std_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
        chem_nodes = [n for n in g.nodes if '>>' not in n]
        starting_nodes = [n for n in chem_nodes if g.nodes[n]['terminal']]
        s_p_g = metabolic_dijkstra(g, starting_nodes)
        scs_spg_dict[smiles][prioritizer] = s_p_g
        try:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[std_smiles]['path_length']
        except:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[smiles]['path_length']
    else:
        print ('No Graph for', smiles, prioritizer)
except:
    print ('Graph retrieval failed for', smiles)

In [115]:
scs_spg_dict[smi]['async'].nodes[smi]

{'type': 'chemical',
 'score': 112.76640682967036,
 'sfscore': [0.5027291774749756, 0.5171467661857605],
 'chem_score': 56.07544387738171,
 'enzy_score': 54.44962420331848,
 'depth': -1,
 'terminal': False,
 'path_length': 4,
 'visited': False,
 'shortest_pathway': ['CC(C)[C@@H](C)N>>CC[C@@H](C)N',
  'CC[C@@H](C)N.O=C(Cl)CCl>>CC[C@@H](C)NCC(=O)N[C@H](C)CC',
  'C1CCOC1.CC[C@@H](C)NCC(=O)N[C@H](C)CC.[Li+].[AlH4-]>>CC[C@@H](CO)NCCN[C@H](C)CC',
  'CC[C@@H](CO)NCCN[C@H](C)CC>>CC[C@@H](CO)NCCN[C@@H](CC)CO']}

In [117]:
# Path 20 shown below is manually checked and selected
max_depth = 5
max_trees = 5000
validate_paths = True
NIL_UUID = '00000000-0000-0000-0000-000000000000'

smiles = smi
prioritizer = 'async'

paths = get_paths(
                scs_spg_dict[smiles][prioritizer].reverse(),
                root=smiles,
                root_uuid=NIL_UUID,
                max_depth=max_depth,
                max_trees=max_trees,
                validate_paths=validate_paths,
            )
all_paths = nx_paths_to_json(paths, NIL_UUID)

set_path_length = 5
n = 0
for path in all_paths:
    rxn_list = get_all_children(path)
    path_length = len(rxn_list)
    if path_length == set_path_length:
        print(f'Path {n}:')
        print_rxns_list(scs_spg_dict,prioritizer,smiles,rxn_list)
        n+=1

Path 0:
Product SFSCore: [0.503 0.517] ['bkms'] :  CC[C@@H](CO)NCCN[C@H](C)CC>>CC[C@@H](CO)NCCN[C@@H](CC)CO


Product SFSCore: [0.537 0.471] ['RXN4Chem'] :  C1CCOC1.CC[C@@H](C)NCC(=O)N[C@H](C)CC.[Li+].[AlH4-]>>CC[C@@H](CO)NCCN[C@H](C)CC


Product SFSCore: [0.52  0.494] ['RXN4Chem'] :  CC[C@@H](C)N.O=C(Cl)CCl>>CC[C@@H](C)NCC(=O)N[C@H](C)CC


Product SFSCore: [0.496 0.526] ['bkms'] :  CC[C@H](N)CN>>CC[C@@H](C)N


Product SFSCore: [0.529 0.481] ['RXN4Chem'] :  C1CCOC1.CC[C@@H](N)C(N)=O.[Li+].[AlH4-]>>CC[C@H](N)CN


...

Path 31:
Product SFSCore: [0.503 0.517] ['bkms'] :  CC[C@@H](CO)NCCN[C@@H](CC)CCl>>CC[C@@H](CO)NCCN[C@@H](CC)CO


Product SFSCore: [0.553 0.451] ['bkms'] :  CC[C@@H](CCl)NCCN[C@H](C)CC>>CC[C@@H](CO)NCCN[C@@H](CC)CCl


Product SFSCore: [0.575 0.426] ['RXN4Chem'] :  C1CCOC1.CC[C@@H](C)NCC(=O)N[C@H](C)CC.[Li+].[AlH4-]>>CC[C@@H](CCl)NCCN[C@H](C)CC


Product SFSCore: [0.52  0.494] ['RXN4Chem'] :  CC[C@@H](C)N.O=C(Cl)CCl>>CC[C@@H](C)NCC(=O)N[C@H](C)CC


Product SFSCore: [0.496 0.526] ['RXN4Chem'] :  CC/C(C)=N/O.[H][H]>>CC[C@@H](C)N


# Case study: Epidiolex

In [111]:
scs_shortest_path_dict = {}
scs_spg_dict = {}

smi = 'C=C(C)[C@@H]1CCC(C)=C[C@H]1c1c(O)cc(CCCCC)cc1O'
name = 'Epidiolex'
path = f'./results/case_study/async_{name}.json'
with open(path, "r") as outfile:
    result_dict = json.load(outfile)


smiles = result_dict['start_node']['id']
scs_shortest_path_dict[smiles] = {}
scs_spg_dict[smiles] = {}
prioritizer = 'async'

graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
try:
    g = nx.node_link_graph(graph_json)
    if g and len(g.nodes)>1:
        std_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
        chem_nodes = [n for n in g.nodes if '>>' not in n]
        starting_nodes = [n for n in chem_nodes if g.nodes[n]['terminal']]
        s_p_g = metabolic_dijkstra(g, starting_nodes)
        scs_spg_dict[smiles][prioritizer] = s_p_g
        try:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[std_smiles]['path_length']
        except:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[smiles]['path_length']
    else:
        print ('No Graph for', smiles, prioritizer)
except:
    print ('Graph retrieval failed for', smiles)

In [112]:
scs_spg_dict[smi]['async'].nodes[smi]

{'type': 'chemical',
 'score': 243.30518852908415,
 'sfscore': [0.4377802312374115, 0.6024901866912842],
 'chem_score': 136.79098683355969,
 'enzy_score': 96.71620006923814,
 'depth': -1,
 'terminal': False,
 'path_length': 2,
 'visited': False,
 'shortest_pathway': ['CC(C)=CCC/C(C)=C/CBr.CCCCCc1cc(O)cc(O)c1>>CCCCCc1cc(O)c(C/C=C(\\C)CCC=C(C)C)c(O)c1',
  'CCCCCc1cc(O)c(C/C=C(\\C)CCC=C(C)C)c(O)c1>>C=C(C)[C@@H]1CCC(C)=C[C@H]1c1c(O)cc(CCCCC)cc1O']}

In [113]:
# Path 1 shown below is manually checked and selected
max_depth = 4
max_trees = 5000
validate_paths = True
NIL_UUID = '00000000-0000-0000-0000-000000000000'

smiles = smi
prioritizer = 'async'

paths = get_paths(
                scs_spg_dict[smiles][prioritizer].reverse(),
                root=smiles,
                root_uuid=NIL_UUID,
                max_depth=max_depth,
                max_trees=max_trees,
                validate_paths=validate_paths,
            )
all_paths = nx_paths_to_json(paths, NIL_UUID)

set_path_length = 2
n = 0
for path in all_paths:
    rxn_list = get_all_children(path)
    path_length = len(rxn_list)
    if path_length == set_path_length:
        print(f'Path {n}:')
        print_rxns_list(scs_spg_dict,prioritizer,smiles,rxn_list)
        n+=1

Path 0:
Product SFSCore: [0.438 0.602] ['bkms'] :  CCCCCc1cc(O)c(C/C=C(\C)CCC=C(C)C)c(O)c1>>C=C(C)[C@@H]1CCC(C)=C[C@H]1c1c(O)cc(CCCCC)cc1O


Product SFSCore: [0.437 0.607] ['RXN4Chem'] :  CC(C)=CCC/C(C)=C/CBr.CCCCCc1cc(O)cc(O)c1>>CCCCCc1cc(O)c(C/C=C(\C)CCC=C(C)C)c(O)c1


Path 1:
Product SFSCore: [0.438 0.602] ['bkms'] :  CCCCCc1cc(O)c(C/C=C(\C)CCC=C(C)C)c(O)c1>>C=C(C)[C@@H]1CCC(C)=C[C@H]1c1c(O)cc(CCCCC)cc1O


Product SFSCore: [0.437 0.607] ['RXN4Chem'] :  CC(C)=CCC/C(C)=C/CO.CCCCCc1cc(O)cc(O)c1>>CCCCCc1cc(O)c(C/C=C(\C)CCC=C(C)C)c(O)c1


# Case study: Rivastigmine

In [10]:
scs_shortest_path_dict = {}
scs_spg_dict = {}

smi = 'CCN(C)C(=O)Oc1cccc([C@@H](C)O)c1'
name = 'Rivastigmine-44'
path = f'./results/case_study/async_{name}.json'
with open(path, "r") as outfile:
    result_dict = json.load(outfile)


smiles = result_dict['start_node']['id']
scs_shortest_path_dict[smiles] = {}
scs_spg_dict[smiles] = {}
prioritizer = 'async'

graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
try:
    g = nx.node_link_graph(graph_json)
    if g and len(g.nodes)>1:
        std_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
        chem_nodes = [n for n in g.nodes if '>>' not in n]
        starting_nodes = [n for n in chem_nodes if g.nodes[n]['terminal']]
        s_p_g = metabolic_dijkstra(g, starting_nodes)
        scs_spg_dict[smiles][prioritizer] = s_p_g
        try:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[std_smiles]['path_length']
        except:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[smiles]['path_length']
    else:
        print ('No Graph for', smiles, prioritizer)
except:
    print ('Graph retrieval failed for', smiles)

In [11]:
scs_spg_dict[smi]['async'].nodes[smi]

{'type': 'chemical',
 'score': 130.0,
 'sfscore': [0.529366135597229, 0.482090562582016],
 'chem_score': 61.18240237236023,
 'enzy_score': 67.32822686433792,
 'depth': -1,
 'terminal': False,
 'path_length': 1,
 'visited': False,
 'shortest_pathway': ['CCN(C)C(=O)Oc1cccc(C(C)=O)c1.CO.[BH4-].[Na+]>>CCN(C)C(=O)Oc1cccc([C@@H](C)O)c1']}

In [12]:
max_depth = 1
max_trees = 5000
validate_paths = True
NIL_UUID = '00000000-0000-0000-0000-000000000000'

smiles = smi
prioritizer = 'async'

paths = get_paths(
                scs_spg_dict[smiles][prioritizer].reverse(),
                root=smiles,
                root_uuid=NIL_UUID,
                max_depth=max_depth,
                max_trees=max_trees,
                validate_paths=validate_paths,
            )
all_paths = nx_paths_to_json(paths, NIL_UUID)

set_path_length = 1
n = 0
for path in all_paths:
    rxn_list = get_all_children(path)
    path_length = len(rxn_list)
    if path_length == set_path_length:
        print(f'Path {n}:')
        print_rxns_list(scs_spg_dict,prioritizer,smiles,rxn_list)
        n+=1

Path 0:
Product SFSCore: [0.529 0.482] ['RXN4Chem'] :  CCN(C)C(=O)Oc1cccc(C(C)=O)c1.CO.[BH4-].[Na+]>>CCN(C)C(=O)Oc1cccc([C@@H](C)O)c1


Path 1:
Product SFSCore: [0.529 0.482] ['bkms'] :  CCN(C)C(=O)Oc1cccc(C(C)=O)c1>>CCN(C)C(=O)Oc1cccc([C@@H](C)O)c1


# Case study: Formoterol

In [13]:
scs_shortest_path_dict = {}
scs_spg_dict = {}

smi = 'O=[N+]([O-])c1cccc([C@@H]2CO2)c1'
name = 'formoterol-46'
path = f'./results/case_study/async_{name}.json'
with open(path, "r") as outfile:
    result_dict = json.load(outfile)


smiles = result_dict['start_node']['id']
scs_shortest_path_dict[smiles] = {}
scs_spg_dict[smiles] = {}
prioritizer = 'async'

graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
try:
    g = nx.node_link_graph(graph_json)
    if g and len(g.nodes)>1:
        std_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
        chem_nodes = [n for n in g.nodes if '>>' not in n]
        starting_nodes = [n for n in chem_nodes if g.nodes[n]['terminal']]
        s_p_g = metabolic_dijkstra(g, starting_nodes)
        scs_spg_dict[smiles][prioritizer] = s_p_g
        try:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[std_smiles]['path_length']
        except:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[smiles]['path_length']
    else:
        print ('No Graph for', smiles, prioritizer)
except:
    print ('Graph retrieval failed for', smiles)

In [14]:
scs_spg_dict[smi]['async'].nodes[smi]

{'type': 'chemical',
 'score': 90.33459118601274,
 'sfscore': [0.5742165446281433, 0.42704617977142334],
 'chem_score': 38.46297437478458,
 'enzy_score': 51.75754911881271,
 'depth': -1,
 'terminal': False,
 'path_length': 1,
 'visited': False,
 'shortest_pathway': ['C=Cc1cccc([N+](=O)[O-])c1.ClCCl.O=C(OO)c1cccc(Cl)c1>>O=[N+]([O-])c1cccc([C@@H]2CO2)c1']}

In [15]:
# Path 0 shown below is manually checked and selected
max_depth = 1
max_trees = 5000
validate_paths = True
NIL_UUID = '00000000-0000-0000-0000-000000000000'

smiles = smi
prioritizer = 'async'

paths = get_paths(
                scs_spg_dict[smiles][prioritizer].reverse(),
                root=smiles,
                root_uuid=NIL_UUID,
                max_depth=max_depth,
                max_trees=max_trees,
                validate_paths=validate_paths,
            )
all_paths = nx_paths_to_json(paths, NIL_UUID)

set_path_length = 1
n = 0
for path in all_paths:
    rxn_list = get_all_children(path)
    path_length = len(rxn_list)
    if path_length == set_path_length:
        print(f'Path {n}:')
        print_rxns_list(scs_spg_dict,prioritizer,smiles,rxn_list)
        n+=1

Path 0:
Product SFSCore: [0.574 0.427] ['RXN4Chem'] :  C=Cc1cccc([N+](=O)[O-])c1.ClCCl.O=C(OO)c1cccc(Cl)c1>>O=[N+]([O-])c1cccc([C@@H]2CO2)c1


Path 1:
Product SFSCore: [0.574 0.427] ['bkms'] :  C=Cc1cccc([N+](=O)[O-])c1>>O=[N+]([O-])c1cccc([C@@H]2CO2)c1


Path 2:
Product SFSCore: [0.574 0.427] ['bkms'] :  c1ccc([C@@H]2CO2)cc1>>O=[N+]([O-])c1cccc([C@@H]2CO2)c1


In [19]:
scs_shortest_path_dict = {}
scs_spg_dict = {}

smi = 'COc1ccc(C[C@@H](C)N)cc1'
name = 'formoterol-48'
path = f'./results/case_study/async_{name}.json'
with open(path, "r") as outfile:
    result_dict = json.load(outfile)


smiles = result_dict['start_node']['id']
scs_shortest_path_dict[smiles] = {}
scs_spg_dict[smiles] = {}
prioritizer = 'async'

graph_json = build_graph_from_async(result_dict['explored_rxns'],result_dict['explored_nodes'],result_dict['start_node'])
try:
    g = nx.node_link_graph(graph_json)
    if g and len(g.nodes)>1:
        std_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
        chem_nodes = [n for n in g.nodes if '>>' not in n]
        starting_nodes = [n for n in chem_nodes if g.nodes[n]['terminal']]
        s_p_g = metabolic_dijkstra(g, starting_nodes)
        scs_spg_dict[smiles][prioritizer] = s_p_g
        try:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[std_smiles]['path_length']
        except:
            scs_shortest_path_dict[smiles][prioritizer] = s_p_g.nodes[smiles]['path_length']
    else:
        print ('No Graph for', smiles, prioritizer)
except:
    print ('Graph retrieval failed for', smiles)

In [20]:
scs_spg_dict[smi]['async'].nodes[smi]

{'type': 'chemical',
 'score': 85.13843876330611,
 'sfscore': [0.5514477491378784, 0.45322513580322266],
 'chem_score': 38.18903834216786,
 'enzy_score': 46.551558292732345,
 'depth': -1,
 'terminal': False,
 'path_length': 1,
 'visited': False,
 'shortest_pathway': ['CC(=O)O.[NH4+].CO.COc1ccc(CC(C)=O)cc1.[BH3-]C#N.[Na+]>>COc1ccc(C[C@@H](C)N)cc1']}

In [21]:
# Path 1 shown below is manually checked and selected. Another reagent, (+)-Phenylethylamine, is extracted from literature found by SciFinder.
max_depth = 1
max_trees = 5000
validate_paths = True
NIL_UUID = '00000000-0000-0000-0000-000000000000'

smiles = smi
prioritizer = 'async'

paths = get_paths(
                scs_spg_dict[smiles][prioritizer].reverse(),
                root=smiles,
                root_uuid=NIL_UUID,
                max_depth=max_depth,
                max_trees=max_trees,
                validate_paths=validate_paths,
            )
all_paths = nx_paths_to_json(paths, NIL_UUID)

set_path_length = 1
n = 0
for path in all_paths:
    rxn_list = get_all_children(path)
    path_length = len(rxn_list)
    if path_length == set_path_length:
        print(f'Path {n}:')
        print_rxns_list(scs_spg_dict,prioritizer,smiles,rxn_list)
        n+=1

Path 0:
Product SFSCore: [0.551 0.453] ['RXN4Chem'] :  CC(=O)O.[NH4+].CO.COc1ccc(CC(C)=O)cc1.[BH3-]C#N.[Na+]>>COc1ccc(C[C@@H](C)N)cc1


Path 1:
Product SFSCore: [0.551 0.453] ['RXN4Chem'] :  COc1ccc(CC(C)=O)cc1.N.[H][H]>>COc1ccc(C[C@@H](C)N)cc1


Path 2:
Product SFSCore: [0.551 0.453] ['bkms'] :  COc1ccc(CCCN)cc1>>COc1ccc(C[C@@H](C)N)cc1


Path 3:
Product SFSCore: [0.551 0.453] ['bkms'] :  COc1ccc(CC(C)=O)cc1>>COc1ccc(C[C@@H](C)N)cc1
